# Modelos y Simulaciones

### Importamos librerias

In [177]:
import numpy as np
import random
import math
import queue

### Generamos valores aleatorios

In [191]:
## Obtiene una velocidad probabilistica para cargar pizzas
def velocidad_carga_pizza():
    return np.random.exponential(1/10)

## Obtiene un tiempo de entrega probabilistico
def tiempo_entrega():
    return np.random.exponential(1/10)

## Obtiene si se convenció al cliente o no de cambiar el tipo de pizza
def cliente_convencido():
    return np.random.binomial(1, 0.3) == 1

## Obtiene pedidos generados en una hora
def pedidos_generados():
    return np.random.poisson(20)

## Obtiene el minuto  de un pedido en una hora
def pedido_en_hora():
    return math.trunc(random.uniform(0,60))

## Genera tipo de pizza aleatorio
## se tiene que asegurar que la nueva pizza no sea la misma que ya pidio
def generar_tipo_de_pizza():
    opcion = random.random()
    if(opcion < 0.05):
        return 'anana'
    elif(opcion < 0.20):
        return 'calabresa'
    elif(opcion < 0.55):
        return 'mozzarella'
    elif(opcion < 0.75):
        return 'fugazzeta'
    else:
        return 'napolitana'
    
## Obtiene una ubicación del cliente aleatoria
def generar_ubicacion_cliente():
    ubicacion = np.random.normal(0, 10, 2)
    return ubicacion * 100 ##Para que quede como maximo 2000 como en el gráfico de bruno

In [192]:
for i in range(10):
    print(generar_ubicacion_cliente())

[1150.69494897 1110.1149252 ]
[531.85612118  83.73732475]
[  571.44708359 -1717.04071483]
[ -747.90054739 -1252.85613447]
[  48.53890452 -722.39441768]
[1292.40441752 1675.31636187]
[-1557.43778419   661.44425466]
[-976.47034005  926.1343431 ]
[  434.11846524 -1012.3981857 ]
[1019.19173017 -417.26263417]


### Generamos eventos

#### Evento de cliente llama y pide una pizza

In [179]:
class LlamoClienteEvent:
    limite = 2000
    def __init__(self, hora):
        self.hora = pedido_en_hora() + hora * 60
        self.tipo = generar_tipo_de_pizza()
        self.ubicacion = generar_ubicacion_cliente()
    
    def cliente_esta_en_rango(self):
        cateto1 = self.ubicacion[0]
        cateto2 = self.ubicacion[1]
        ##calculamos hipotenusa
        hipotenusa = math.sqrt(math.pow(cateto1) + math.pow(cateto2))
        return hipotenusa <= self.limite
        
    def obtener_camionetas_disponibles(self, camionetas):
        return [camioneta for camioneta in camionetas if camioneta.disponible]
    
    def camionetas_con_pizza_pedida(self, tipo, camionetas):
        return [camioneta for camioneta in camionetas if camioneta.tiene_tipo(tipo)]
    
    def obtener_distancia(punto1, punto2):
        cateto1 = punto2[0] - punto1[0]
        cateto2 = punto2[1] - punto1[1]
        ##calculamos distancia
        return math.sqrt(math.pow(cateto1) + math.pow(cateto2))
    
    def obtener_camioneta_mas_cercana(self, ubicacion, camionetas):
        #obtenemos distancia entre las camionetas y la ubicacion del pedido
        distancias = map(lambda camioneta: (self.obtener_distancia(camioneta.ubicacion, self.ubicacion), camioneta), camionetas)
        #obtenemos la minima distancia
        distancia_minima = min(distancias, key=lambda distancia: distancia[0])
        #obtenemos camioneta
        return distancia_minima[1]
    
    ##REFACTORIZAR
    def ejecutarActividad(self, dia):
        if(self.cliente_esta_en_rango()):
            camionetas_disponibles = self.obtener_camionetas_disponibles(dia.camionetas)
            if(len(camionetas_disponibles) == 0):
                dia.encolar_cliente(self)
            else:
                camionetas_posibles_de_envio = self.camionetas_con_pizza_pedida(self.tipo, camionetas_disponibles)
                if (len(camionetas_posibles_de_envio) > 0):
                    camioneta = self.obtener_camioneta_mas_cercana(self.ubicacion, camionetas_posibles_de_envio)
                    enviar_pedido(camioneta, dia)
                else:
                    if (cliente_convencido()):
                        tipo_pizza = generar_tipo_de_pizza()
                    else:
                        camioneta = obtener_camioneta_mas_cercana([0,0], camionetas_disponibles)
                        cargar_camioneta(camioneta)
        else: 
            self.rechazar_pedido(dia)
 
    def rechazar_pedido(self, dia):
        dia.pedidos_rechazados += 1
        

#### Evento de vencimiento de pizza

In [180]:
class PizzaVenceEvent:
    tiempo_vencimiento_de_pizza = 120
    
    def __init__(self, pizza, camioneta):
        self.hora = pizza.hora + self.tiempo_vencimiento_de_pizza
        self.pizza = pizza
        self.camioneta = camioneta

    def ejecutarActividad(self, dia):
        self.camioneta.quitar_pizza(self.pizza)
        dia.desperdicio += 1

In [173]:
generar_ubicacion_cliente()

array([-15.05889324,  32.8773531 ])

3 Formas de realizar los calculos finales de la simulacion.

1. Directamente hacer sumatorias de las cosas. 
2. Arreglo donde cada columna es los datos de un dia y cada fila es un dia
3. Arreglo de Dias.

In [79]:
class Pizza:
    def __init__(self, tipo, hora):
        self.tipo = tipo
        self.hora = hora

class Camioneta:
    pizzas_maximas = 40
    def __init__(self):
        self.ubicacion = [0, 0]
        self.pizzas = []
        self.disponible = True
        
    def cargar_camionetas(self, hora, fel):
        cantidad_pizzas_a_cargar = pizzas_maximas - len(self.pizzas)
        for i in range(cantidad_pizzas_a_cargar):
            pizza = self.generar_pizza(hora, fel)
            self.pizzas.append(pizza)
    
    def generar_pizza(self, hora, fel):
        tipo = generar_tipo_de_pizza()
        pizza = Pizza(tipo, hora)
        self.generar_evento_de_vencimiento(pizza, fel)
        return pizza
        
    def generar_evento_de_vencimiento(self, pizza, fel):
        vencimiento_de_pizza = PizzaVenceEvent(pizza, self)
        fel.append(vencimiento_de_pizza)

    def quitar_pizza(self, pizza):
        self.pizzas.remove(pizza)
        
    def tiene_tipo(self, tipo):
        lista_de_pizzas_de_tipo = [pizza for pizza in self.pizzas if pizza.tipo == tipo]
        return len(lista_de_pizzas_de_tipo) > 0
    
class Simulacion:
    experimentos = 10
    dias_a_simular = 365
    horas_por_dia = 12
    
    def __init__():
        minutos_maximo = 60 * horas_por_dia
        self.camionetas = [Camioneta(), Camioneta(),Camioneta(), Camioneta()]

class Dia:
    def __init__(self, camionetas):
        self.camionetas = camionetas
        self.tiempo_actual = 0
        self.pedidos_rechazado = 0
        self.cola_espera_clientes = queue.Queue()
        self.desperdicios = 0
        
    def iniciar_dia(self):
        tiempo_actual = 0
        self.fel = generar_pedidos()
        self.ubicar_camionetas()
        self.cargar_camionetas()
    
    def ubicar_camionetas(self):
        for camioneta in self.camionetas:
            camioneta.ubicacion = [0, 0]

    def generar_pedidos_en_hora(self, hora):
        eventos_de_pedidos = []
        for i in range(pedidos_generados()): 
            llamaClienteEvent = LlamoClienteEvent()
            eventos_de_pedidos.append(llamaClienteEvent)
        return eventos_de_pedidos

    def generar_pedidos(self):
        fel = []
        for i in range(12):
            fel += generar_pedidos_en_hora(i)
        return fel
            
    def cargar_camionetas(self):
        for camioneta in self.camionetas:
            camioneta.cargar_pizzas(self.tiempo_actual, self.fel)
    
    def encolar_cliente(self, cliente):
        self.cola_espera_clientes.put(cliente)
    
    def termino_dia(self):
        return tiempo_actual > minutos_maximo
    
    def acciones_finales_del_dia(self):
        limpiar_pizzas_en_camionetas(self.camionetas)
    
    ## OBTENER CLIENTE CON ESTA FUNCION
    def obtener_cliente_de_cola():
        if not self.cola_espera_clientes.empty():
            return self.cola_espera_clientes.get()
        else
            return None

In [1]:
from .Simulacion import Simulacion

ModuleNotFoundError: No module named '__main__.Simulacion'; '__main__' is not a package